**Ejercicio 2:** Diseñe e implemente un algoritmo genético que busque el mejor subconjunto de características para la clasificación de cáncer (leucemia linfocítica aguda y leucemia mielógena aguda) a partir de datos genómicos. Se proveen 38 muestras en el conjunto de entrenamiento y 34 en el conjunto de prueba (leukemia train.csv y leukemia test.csv, respectivamente). Cada muestra se compone de un total de 7129 características, que corresponden a valores de expresión génica

#### **Librerías**

In [84]:
import random as rdm
import numpy as np
import warnings
import sklearn.svm as svm
import sklearn.metrics as metrics
from sklearn import metrics, svm
import matplotlib.pyplot as plt
from sklearn.neural_network import MLPClassifier

warnings.filterwarnings("ignore") # Se desactivan warnings de conversion de datos


<h4><b>Algoritmo</b></h2>


In [85]:
# ESTA FUNCION NO HACE FALTA YA QUE LA EVALUACION CONVIENE HACERLA EN BINARIO NOMAS

def decodif(poblacion):     # Pasa la población del genotipo (binario) al fenotipo (decimal)
    poblacion_dec = []
    for genotipo in poblacion:
        decimal = int(genotipo[1:-4], 2) + 0.1*int(genotipo[-4:],2)   # Convierte la cadena binaria a un entero
        if genotipo[0] == '1':
            poblacion_dec.append(-decimal)
        else:
            poblacion_dec.append(decimal)
    return poblacion_dec

In [86]:
def competencias(aptitudes,k=2):
    competidores = np.random.randint(0,len(aptitudes),(2,1))
    ganador = np.argmax(aptitudes[competidores])
    return ganador

In [87]:
def cruza(padre,madre,prob):
    prob_i = np.random.rand()
    if prob_i < prob:
        punto_corte = rdm.randint(1,len(padre))
        hijo1 = padre[:punto_corte] + madre[punto_corte:]
        hijo2 = madre[:punto_corte] + padre[punto_corte:]
        #hijo1 = np.concatenate((padre[:punto_corte], madre[punto_corte:]))
        #hijo2 = np.concatenate((madre[:punto_corte], padre[punto_corte:]))
    else:
        hijo1 = padre
        hijo2 = madre
    return hijo1,hijo2

def mutacion(individuo,prob):   # Le paso probabilidad a nivel de individuo
    prob_i = np.random.rand()
    if prob_i < prob:
        pos = np.random.choice(len(individuo))
        individuo_aux = list(individuo)
        if(individuo_aux[pos] == '0'):
            individuo_aux[pos] = '1'
        else:
            individuo_aux[pos] = '0'

        individuo = ''.join(individuo_aux) 
    return individuo

In [88]:
def elitismo(poblacion,aptitudes):
    apt_aux = list(enumerate(aptitudes))
    ordenados = sorted(apt_aux,key=lambda x: x[1], reverse=True)
    idx1 = ordenados[0][0]  # Mayor aptitud
    idx2 = ordenados[1][0]  # Segunda mayor aptitud
    return poblacion[idx1],poblacion[idx2]  # Devuelve los índices de los individuos 

In [89]:
def algoritmo_genetico(size_p,bits,max_gen,max_estable,func_aptitud,prob_mutacion,prob_cruza,train_input,train_yd,monitor_input,monitor_yd,alpha,beta):

    v_fitness_completo = []
    # PASO 1: Inicialización de la población
    poblacion = []
    for i in range(size_p):
        ind_aux = ['0' for _ in range(bits)]
        numeros_aleatorios = rdm.sample(range(bits - 1), 27)
        for j in numeros_aleatorios:
            aux = rdm.random()
            if aux < 0.90:
                ind_aux[j] = '1'
        poblacion.append(''.join(ind_aux))
    #print(poblacion[0])
    #for individuo in poblacion:
    #    if int(individuo[-4],2) > 9:
    #        individuo = individuo[:-4] + '1001' # Que no pase de 9 la parte decimal
            
    cont_estabilidad = 0
    apt_max = 1e-6

    #print(len(poblacion),',',len(poblacion[0]))
    
    for gen in range(max_gen):
        #poblacion_dec = decodif(poblacion)        # Decodifico para pasarselo a la función de aptitud
        aptitudes = func_aptitud(poblacion, train_input, train_yd, monitor_input, monitor_yd, alpha, beta)
        nueva_pob = []
        v_fitness_completo = np.concatenate((v_fitness_completo, aptitudes), axis=0)
        # Elitismo
        mejor1,mejor2 = elitismo(poblacion,aptitudes)
        nueva_pob.append(mejor1)
        nueva_pob.append(mejor2)
        
        while(len(nueva_pob) < len(poblacion)):
            # PASO 2: Selección (elegimos uno de los métodos propuestos en funciones auxiliares):
            padre = poblacion[competencias(aptitudes,4)]
            madre = poblacion[competencias(aptitudes,4)]
            # PASO 3: CRUZA (me va a devolver los hijos cruzados o los padres, depende probabilidad)
            hijo1, hijo2 = cruza(padre,madre,prob_cruza)               
            # PASO 4: MUTACION (me va a devolver hijos mutados o no, depende probabilidad)  
            hijo1 = mutacion(hijo1,prob_mutacion)   
            hijo2 = mutacion(hijo2,prob_mutacion)
            # Los agrego a la nueva generación      
            nueva_pob.append(hijo1)
            nueva_pob.append(hijo2)

        # PASO 5: EVALUACIÓN (vemos si se estabilizó por elitismo)
        if(max(aptitudes) == apt_max):
            cont_estabilidad += 1
        else:
            cont_estabilidad = 0
        if(cont_estabilidad >= max_estable):
            break

        poblacion = nueva_pob
        apt_max = max(aptitudes)

    #poblacion_dec = decodif(poblacion)
    print('El algoritmo terminó en la generación',gen)
    apt_aux = list(enumerate(aptitudes))
    ordenados = sorted(apt_aux,key=lambda x: x[1], reverse=True)
    idx = ordenados[0][0]
    return poblacion[idx], v_fitness_completo

In [90]:
def func_aptitud(poblacion, train_input, train_yd, monitor_input, monitor_yd, alpha, beta):
    v_fitness = np.zeros(len(poblacion))
    
    for i, cadena in enumerate(poblacion):
        v_aux = []
        for j, bit in enumerate(cadena):
            if bit == '1':
                v_aux.append(j)
        #print(v_aux)
        new_train_input = train_input[:, v_aux]
        new_monitor_input = monitor_input[:, v_aux]
        
        clf = svm.SVC()
        clf.fit(new_train_input, train_yd)      # ya hace monitoreo solo, podemos sacar esa parte
        y_pred = clf.predict(new_monitor_input)
        
        fitness = metrics.f1_score(np.ravel(monitor_yd), y_pred)    
        
        long_t = len(train_input[0])
        long_g = len(new_train_input[0])
        
        #print(alpha * fitness, beta * (long_g / long_t))
        
        v_fitness[i] = alpha * fitness - beta * (long_g / long_t)
    #for _ in range(len(v_fitness)):
        #print(v_fitness)
    return v_fitness


In [91]:
#--- Parámetros del modelo ---#
size_p = 20
bits = 7129
max_gen = 1000
max_estable = 50
prob_mutacion = 0.1
prob_cruza = 0.7
alpha = 0.9
beta = 0.1

#--- Carga de datos ---#
train_data = np.loadtxt('./data/leukemia_train.csv', delimiter=',')
test_data = np.loadtxt('./data/leukemia_test.csv', delimiter=',')
monitor_data = np.empty((0, train_data.shape[1]))  # Inicializa monitor_data como una matriz vacía

#--- Separación de datos para monitoreo ---#
n_monitor = 10
for i in range(n_monitor):
    idx = rdm.randint(0, train_data.shape[0] - 1)              # Determina una fila aleatoria para pasar a monitorio
    monitor_data = np.vstack([monitor_data, train_data[idx]])  # Agrega la fila al monitoreo
    train_data = np.delete(train_data, idx, axis=0)            # Elmina la fila de los datos de train

#--- Separación de la data en entradas y salidas ---#
train_input = train_data[:, :-1]
train_yd = train_data[:, -1].reshape(-1, 1)
monitor_input = monitor_data[:, :-1]
monitor_yd = monitor_data[:, -1].reshape(-1, 1)
test_input = test_data[:, :-1]
test_yd = test_data[:, -1].reshape(-1, 1)

ganador, vector_fitness_completo = algoritmo_genetico(size_p,bits,max_gen,max_estable,func_aptitud,prob_mutacion,prob_cruza,train_input,train_yd,monitor_input,monitor_yd,alpha,beta)
print(ganador)

#plt.plot(vector_fitness_completo)


El algoritmo terminó en la generación 84
00000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000100000000000000000000000000000000000000000000000000000000000000000000000001000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000001000000000000000000000000000000000000000000000000000001000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000010000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000

In [92]:
v_indices = []
for i in range (len(ganador)):
    if(ganador[i] == '1'):
        v_indices.append(i)
print(v_indices)

len(v_indices)

clf = svm.SVC()
clf.fit(train_input[:, v_indices], np.ravel(train_yd))
y_pred = clf.predict(test_input[:, v_indices])

fitness = metrics.f1_score(np.ravel(test_yd), y_pred)
print(fitness)

[113, 187, 337, 391, 645, 991, 1776, 1876, 2042, 2572, 3074, 3558, 4286, 5266, 5657, 5993]
0.3333333333333333


In [93]:
clf = svm.SVC()
clf.fit(train_input, np.ravel(train_yd))
y_pred = clf.predict(test_input)

fitness = metrics.f1_score(np.ravel(test_yd), y_pred)
print(fitness)

0.0
